In [ ]:
import pandas as pd

file_path = "MasterDataset_WithWeatherAndEvent.csv"

data = pd.read_csv(file_path)

C:\Users\amyna\AppData\Local\Temp\ipykernel_20092\2259706258.py:5: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path)


In [ ]:
data.head()

#first two columns look like index positions so I will be removing them


In [ ]:
data = data.drop(data.columns[[0,1]], axis=1)
data = data.drop(["date", "time"], axis=1)

In [ ]:
data['holiday'].fillna("Not a holiday", inplace=True)
data = data.dropna(how='any',axis=0)

In [ ]:
data.head()

In [ ]:
data[['Start date', 'End date']] = data[['Start date', 'End date']].apply(pd.to_datetime)
data[['Start station', 'Start station number', 'End station','End station number', 'Member type', 'holiday']] = data[['Start station', 'Start station number', 'End station','End station number', 'Member type', 'holiday']].astype('category')
data[["Duration", "temperature_2m", "relativehumidity_2m", "precipitation", "windspeed_10m"]] = data[["Duration", "temperature_2m", "relativehumidity_2m", "precipitation", "windspeed_10m"]].astype('float')
data["isHoliday"] = data["isHoliday"].astype("boolean")

In [ ]:
data[["year", "month", "m_day", "hour"]] = data["Start date"].apply(lambda x: x.timetuple()[0:4]).tolist()
data["w_day"] = data["Start date"].apply(lambda x: x.weekday())

In [ ]:
data["Member type"] = data["Member type"].str.lower()
data = data[data["Start date"].dt.year < 2020]

In [ ]:
non_negative = data['Duration'] >= 0

chunks = []
size = 1000

for element in range (0, len(data), size):
    item = element + size
    segment = data[element:item]
    chunk = segment[non_negative]
    chunks.append(chunk)

filtered = pd.concat(chunks)


In [ ]:
before_outlier = len(data)
before_outlier

In [ ]:
cols = ["Duration"] #Numeric columns only
Q1 = data[cols].quantile(0.25)
Q3 = data[cols].quantile(0.75)
IQR = Q3 - Q1
data = data[~((data[cols] < (Q1 - 1.5 * IQR)) |(data[cols] > (Q3 + 1.5 * IQR))).any(axis=1)]

In [ ]:
after_outlier = len(data)

no_of_outliers = before_outlier - after_outlier
no_of_outliers

In [ ]:
#unable to convert some columns to string. will inspect more. 
mixed_type = data['holiday'].apply(lambda x: type(x) is not str)

true_count = mixed_type[mixed_type].count()
false_count = mixed_type[~mixed_type].count()

print(true_count, false_count)

#all vals are already string so we can leave it as object type

In [ ]:
mixed_member = data['Member type'].apply(lambda x: type(x) is not str)

true_member = mixed_member[mixed_member].count()
false_member = mixed_member[~mixed_member].count()

print(true_member, false_member)

all vals are already string so we can leave it as object type

In [ ]:
mixed_start = data['Start station'].apply(lambda x: type(x) is not str)

true_start = mixed_start[mixed_start].count()
false_start = mixed_start[~mixed_start].count()

print(true_start, false_start)

all vals are already string so we can leave it as object type

In [ ]:
mixed_end = data['End station'].apply(lambda x: type(x) is not str)

true_end = mixed_end[mixed_end].count()
false_end = mixed_end[~mixed_end].count()

print(true_end, false_end)

all vals are already string so we can leave it as object type

In [ ]:
import matplotlib.pyplot as plt

plt.boxplot(data['Duration'])
plt.show()

#Some datapoints above 500000 and 1 below 0. will inspect more. 

In [ ]:
outliers= data[(data['Duration'] < 0) | (data['Duration'] > 500000)]

outliers

For now, we will leave the durations above 500,000 but remove the ones below 0 as negative duration is not possible
Note: Timestamp doesn't contain minute and second information

In [ ]:
Removing non-negative vals due to the size of the dataset

non_negative

    

In [ ]:
#Duration stats

duration_stats = data['Duration'].describe()
duration_stats

In [ ]:
data['Duration'].plot(kind='hist', bins=20, edgecolor='black')
plt.xlabel('Duration')
plt.ylabel('Frequency')
plt.title('Duration Histogram')
plt.ticklabel_format(style='plain', axis = 'y')


In [ ]:
start_station_stats = data['Start station number'].value_counts()
start_station_stats
ss_max = start_station_stats.idxmax()
ss_min = start_station_stats.idxmin()

print(ss_max)
print(ss_min)

In [ ]:
end_station_stats = data['End station number'].value_counts()
end_station_stats
es_max = end_station_stats.idxmax()
es_min = end_station_stats.idxmin()

print(es_max)
print(es_min)

In [ ]:
station_name_1 = data[data['Start station number'] == 31623.0]
station_name_2 = data[data['Start station number'] == 31933.0]
station_name_3 = data[data['End station number'] == 31900.0]

# print(len(station_name_1['Start station']))
# print (len(station_name_2))
print(station_name_3['End station'])

#Most and least busy stations

In [ ]:
data["Member type"] = data["Member type"].str.lower()
member_type = data['Member type'].value_counts()
member_type_counts = member_type.to_dict()

member_type_counts

In [ ]:
plt.pie(member_type, labels=member_type.index, autopct='%1.2f%%', startangle=140)
plt.title ('Member Type')
plt.axis('equal')
plt.show()

In [ ]:
member =member_type_counts['member']/(member_type_counts['member']+member_type_counts['casual'])*100
casual = 100-member

print (casual, "%")
print (member, "%")